# Data Query and Access

## Purpose

This notebook teaches you how to query data from the AM-QADF data warehouse using interactive widgets. You'll learn to use the UnifiedQueryClient to access multiple data sources, apply spatial and temporal filters, and understand the QueryResult structure.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Connect to MongoDB using the infrastructure layer
- ✅ Query data from multiple sources using widgets
- ✅ Apply spatial and temporal filters interactively
- ✅ Understand data structure and format
- ✅ Combine multiple data sources in a single query

## Estimated Duration

45-60 minutes

---

## Overview

The AM-QADF framework provides a unified interface to query data from multiple sources:
- 🛤️ **Hatching Paths**: Layer and path data
- ⚡ **Laser Parameters**: Power, speed, energy density
- 🔬 **CT Scans**: Defect locations and density
- 🌡️ **ISPM Monitoring**: Temperature and sensor measurements
- 📐 **STL Models**: 3D model geometry

Use the interactive widgets below to explore and query data - no coding required!


In [ ]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import AM-QADF and infrastructure
INFRASTRUCTURE_AVAILABLE = False
QUERY_AVAILABLE = False
STL_CLIENT_AVAILABLE = False

try:
    from src.infrastructure.database import get_connection_manager
    INFRASTRUCTURE_AVAILABLE = True
except (ImportError, TypeError, Exception) as e:
    INFRASTRUCTURE_AVAILABLE = False
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}: {e}")

try:
    from am_qadf.query import UnifiedQueryClient, STLModelClient
    from am_qadf.query.base_query_client import QueryResult, SpatialQuery
    QUERY_AVAILABLE = True
    STL_CLIENT_AVAILABLE = True
except ImportError as e:
    QUERY_AVAILABLE = False
    STL_CLIENT_AVAILABLE = False
    print(f"⚠️ Query clients not available: {e}")
    QueryResult = None  # Fallback for demo mode
    SpatialQuery = None  # Fallback for demo mode

# Initialize MongoDB connection
mongo_client = None
unified_client = None
stl_client = None

if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        
        # Test connection
        if mongo_client and mongo_client.is_connected():
            print("✅ MongoDB connection established")
            
            # Test authentication by trying to list collections
            try:
                test_collection = mongo_client.get_collection('stl_models')
                count = test_collection.count_documents({})
                print(f"✅ Authentication verified (found {count} STL models)")
            except Exception as e:
                print(f"⚠️ Authentication test failed: {e}")
                print("   This may indicate authentication issues")
            
            # Initialize query clients
            if QUERY_AVAILABLE:
                try:
                    unified_client = UnifiedQueryClient(mongo_client=mongo_client)
                    print("✅ UnifiedQueryClient initialized")
                except Exception as e:
                    print(f"⚠️ Error initializing UnifiedQueryClient: {e}")
                    import traceback
                    traceback.print_exc()
            
            if STL_CLIENT_AVAILABLE:
                try:
                    stl_client = STLModelClient(mongo_client=mongo_client)
                    print("✅ STLModelClient initialized")
                except Exception as e:
                    print(f"⚠️ Error initializing STLModelClient: {e}")
                    import traceback
                    traceback.print_exc()
        else:
            print("⚠️ MongoDB client not connected")
            print("   Using demo mode with synthetic data")
    except Exception as e:
        print(f"⚠️ MongoDB connection failed: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        print("   Using demo mode with synthetic data")
else:
    print("⚠️ Using demo mode - infrastructure layer unavailable")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ MongoDB connection established
✅ Authentication verified (found 3 STL models)
✅ UnifiedQueryClient initialized
✅ STLModelClient initialized
✅ Setup complete!


## Interactive Data Query Interface

Use the widgets below to query data from the warehouse. Select a model, choose data sources, apply filters, and view results - all interactively!


In [2]:
# Create Interactive Query Interface

# Global state
current_model_id = None
current_model_info = None
query_results = {}
query_dataframes = {}

# ============================================
# Top Panel: Model Selection and Actions
# ============================================

# Get available models
models = []
model_options = [("━━━ Choose a model ━━━", None)]

if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", 
             m.get('model_id')) for m in models
        ])
        if len(model_options) == 1:
            model_options.append(("No models available", None))
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        model_options.append(("Error loading models", None))
else:
    # Demo mode: create synthetic model options
    model_options.extend([
        ("Demo Model 1 (demo-001)", "demo-001"),
        ("Demo Model 2 (demo-002)", "demo-002"),
        ("Demo Model 3 (demo-003)", "demo-003")
    ])

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

query_button = Button(
    description='Query',
    button_style='success',
    icon='search',
    layout=Layout(width='100px')
)

clear_button = Button(
    description='Clear',
    button_style='',
    icon='trash',
    layout=Layout(width='100px')
)

top_panel = HBox([
    model_dropdown,
    query_button,
    clear_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Query Configuration
# ============================================

# Data Source Selection
# Note: get_all_data() queries ALL sources by default, so all checkboxes are True by default
data_source_label = widgets.HTML("<b>Data Sources:</b>")
stl_checkbox = Checkbox(value=True, description='STL', style={'description_width': 'initial'}, layout=Layout(width='auto'))
hatching_checkbox = Checkbox(value=True, description='Hatching', style={'description_width': 'initial'}, layout=Layout(width='auto'))
laser_checkbox = Checkbox(value=True, description='Laser', style={'description_width': 'initial'}, layout=Layout(width='auto'))
ct_checkbox = Checkbox(value=True, description='CT', style={'description_width': 'initial'}, layout=Layout(width='auto'))
ispm_checkbox = Checkbox(value=True, description='ISPM', style={'description_width': 'initial'}, layout=Layout(width='auto'))

data_sources = VBox([
    data_source_label,
    HBox([stl_checkbox, hatching_checkbox]),
    HBox([laser_checkbox, ct_checkbox, ispm_checkbox])
], layout=Layout(padding='5px'))

# Spatial Filter Section
spatial_label = widgets.HTML("<b>Spatial Filter:</b>")
spatial_mode = RadioButtons(
    options=[('Full Model', 'full'), ('Custom Bounding Box', 'custom'), ('Interactive Selection', 'interactive')],
    value='full',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Custom bounding box sliders
bbox_x_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='X Min:', style={'description_width': 'initial'})
bbox_x_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='X Max:', style={'description_width': 'initial'})
bbox_y_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Min:', style={'description_width': 'initial'})
bbox_y_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Max:', style={'description_width': 'initial'})
bbox_z_min = FloatSlider(value=0.0, min=-1000.0, max=1000.0, step=1.0, description='Z Min:', style={'description_width': 'initial'})
bbox_z_max = FloatSlider(value=100.0, min=-1000.0, max=1000.0, step=1.0, description='Z Max:', style={'description_width': 'initial'})

bbox_sliders = VBox([
    bbox_x_min, bbox_x_max,
    bbox_y_min, bbox_y_max,
    bbox_z_min, bbox_z_max
], layout=Layout(display='none'))  # Hidden by default

def update_spatial_controls(change):
    """Show/hide bounding box sliders based on mode."""
    if change['new'] == 'custom':
        bbox_sliders.layout.display = 'flex'
    else:
        bbox_sliders.layout.display = 'none'

spatial_mode.observe(update_spatial_controls, names='value')

spatial_section = VBox([
    spatial_label,
    spatial_mode,
    bbox_sliders
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Temporal Filter Section
temporal_label = widgets.HTML("<b>Temporal Filter:</b>")
temporal_mode = RadioButtons(
    options=[('All', 'all'), ('Layer-based', 'layer'), ('Time-based', 'time')],
    value='all',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Layer range sliders
layer_min = IntSlider(value=0, min=0, max=1000, step=1, description='Layer Min:', style={'description_width': 'initial'})
layer_max = IntSlider(value=100, min=0, max=1000, step=1, description='Layer Max:', style={'description_width': 'initial'})

layer_sliders = VBox([
    layer_min, layer_max
], layout=Layout(display='none'))

# Time range inputs
time_min = FloatText(value=0.0, description='Time Min (s):', style={'description_width': 'initial'})
time_max = FloatText(value=1000.0, description='Time Max (s):', style={'description_width': 'initial'})

time_inputs = VBox([
    time_min, time_max
], layout=Layout(display='none'))

def update_temporal_controls(change):
    """Show/hide temporal controls based on mode."""
    if change['new'] == 'layer':
        layer_sliders.layout.display = 'flex'
        time_inputs.layout.display = 'none'
    elif change['new'] == 'time':
        layer_sliders.layout.display = 'none'
        time_inputs.layout.display = 'flex'
    else:
        layer_sliders.layout.display = 'none'
        time_inputs.layout.display = 'none'

temporal_mode.observe(update_temporal_controls, names='value')

temporal_section = VBox([
    temporal_label,
    temporal_mode,
    layer_sliders,
    time_inputs
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Parameter Filter Section (Collapsible)
parameter_label = widgets.HTML("<b>Parameter Filters:</b>")
parameter_expand = Checkbox(value=False, description='Show Parameter Filters', style={'description_width': 'initial'})

# Laser parameter filters
laser_power_min = FloatSlider(value=0.0, min=0.0, max=1000.0, step=10.0, description='Power Min (W):', style={'description_width': 'initial'})
laser_power_max = FloatSlider(value=1000.0, min=0.0, max=1000.0, step=10.0, description='Power Max (W):', style={'description_width': 'initial'})
scan_speed_min = FloatSlider(value=0.0, min=0.0, max=5000.0, step=50.0, description='Speed Min (mm/s):', style={'description_width': 'initial'})
scan_speed_max = FloatSlider(value=5000.0, min=0.0, max=5000.0, step=50.0, description='Speed Max (mm/s):', style={'description_width': 'initial'})

parameter_sliders = VBox([
    laser_power_min, laser_power_max,
    scan_speed_min, scan_speed_max
], layout=Layout(display='none'))

def update_parameter_controls(change):
    """Show/hide parameter sliders."""
    if change['new']:
        parameter_sliders.layout.display = 'flex'
    else:
        parameter_sliders.layout.display = 'none'

parameter_expand.observe(update_parameter_controls, names='value')

parameter_section = VBox([
    parameter_label,
    parameter_expand,
    parameter_sliders
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Left panel assembly
left_panel = VBox([
    data_sources,
    spatial_section,
    temporal_section,
    parameter_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Results Display
# ============================================

# Results tabs
results_output = Output(layout=Layout(height='600px', overflow='auto'))

# Create tab widgets for different views
table_output = Output()
stats_output = Output()
viz_output = Output()
export_output = Output()

results_tabs = Tab(children=[table_output, stats_output, viz_output, export_output])
results_tabs.set_title(0, 'Table')
results_tabs.set_title(1, 'Statistics')
results_tabs.set_title(2, 'Visualization')
results_tabs.set_title(3, 'Export')

center_panel = VBox([
    widgets.HTML("<h3>Query Results</h3>"),
    results_tabs
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Quick Actions and Status
# ============================================

# Quick actions
quick_actions_label = widgets.HTML("<b>Quick Actions:</b>")
refresh_button = Button(description='Refresh Models', button_style='info', icon='refresh', layout=Layout(width='150px'))
export_button = Button(description='Export Results', button_style='', icon='download', layout=Layout(width='150px'))

quick_actions = VBox([
    quick_actions_label,
    refresh_button,
    export_button
], layout=Layout(padding='5px'))

# Status display
status_label = widgets.HTML("<b>Status:</b>")
status_text = widgets.HTML("Ready to query")
status_display = VBox([
    status_label,
    status_text
], layout=Layout(padding='5px'))

# Results summary
summary_label = widgets.HTML("<b>Results Summary:</b>")
summary_text = widgets.HTML("No query executed yet")
summary_display = VBox([
    summary_label,
    summary_text
], layout=Layout(padding='5px'))

right_panel = VBox([
    quick_actions,
    status_display,
    summary_display
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Progress and Status
# ============================================

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#1f77b4'},
    layout=Layout(width='100%')
)

error_display = widgets.HTML("", layout=Layout(padding='5px'))

bottom_panel = VBox([
    progress_bar,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Query Execution Function
# ============================================

def execute_query(button):
    """Execute query with current widget settings."""
    global current_model_id, query_results, query_dataframes
    
    # Update status
    status_text.value = "<span style='color: blue;'>Executing query...</span>"
    progress_bar.value = 0
    error_display.value = ""
    
    # Get model ID
    model_id = model_dropdown.value
    if not model_id:
        error_display.value = "<span style='color: red;'>⚠️ Please select a model</span>"
        status_text.value = "<span style='color: red;'>Error: No model selected</span>"
        return
    
    current_model_id = model_id
    progress_bar.value = 20
    
    # Get selected data sources
    selected_sources = []
    if stl_checkbox.value:
        selected_sources.append('stl')
    if hatching_checkbox.value:
        selected_sources.append('hatching')
    if laser_checkbox.value:
        selected_sources.append('laser')
    if ct_checkbox.value:
        selected_sources.append('ct')
    if ispm_checkbox.value:
        selected_sources.append('ispm')
    
    if not selected_sources:
        error_display.value = "<span style='color: red;'>⚠️ Please select at least one data source</span>"
        status_text.value = "<span style='color: red;'>Error: No data sources selected</span>"
        progress_bar.value = 0
        return
    
    progress_bar.value = 40
    
    # Get spatial filter
    spatial_bbox = None
    if spatial_mode.value == 'custom':
        spatial_bbox = (
            (bbox_x_min.value, bbox_y_min.value, bbox_z_min.value),
            (bbox_x_max.value, bbox_y_max.value, bbox_z_max.value)
        )
    
    progress_bar.value = 60
    
    # Get temporal filter
    temporal_range = None
    if temporal_mode.value == 'layer':
        # Convert layer range to time (simplified - would need layer-time mapping)
        temporal_range = (layer_min.value * 10.0, layer_max.value * 10.0)  # Assume 10s per layer
    elif temporal_mode.value == 'time':
        temporal_range = (time_min.value, time_max.value)
    
    progress_bar.value = 80
    
    # Execute query
    try:
        if unified_client and mongo_client:
            # Real query - get all data with error handling
            result = {
                'model_id': model_id,
                'stl_model': None,
                'hatching_layers': None,
                'laser_parameters': None,
                'ct_scan': None,
                'ispm_monitoring': None
            }
            
            # Get STL model
            try:
                if unified_client.stl_client:
                    result['stl_model'] = unified_client.stl_client.get_model(model_id)
            except Exception as e:
                result['stl_model'] = {'error': str(e)}
            
            # Get hatching layers
            try:
                if unified_client.hatching_client:
                    layers = unified_client.hatching_client.get_layers(model_id)
                    result['hatching_layers'] = layers if layers else []  # Ensure it's a list, not None
            except Exception as e:
                result['hatching_layers'] = {'error': str(e)}
            
            # Get laser parameters
            try:
                if unified_client.laser_client:
                    spatial_query = SpatialQuery(component_id=model_id)
                    result['laser_parameters'] = unified_client.laser_client.query(spatial=spatial_query)
            except Exception as e:
                result['laser_parameters'] = {'error': str(e)}
            
            # Get CT scan data
            try:
                if unified_client.ct_client:
                    ct_data = unified_client.ct_client.get_scan(model_id)
                    result['ct_scan'] = ct_data  # Can be None if not found
            except Exception as e:
                result['ct_scan'] = {'error': str(e)}
            
            # Get ISPM monitoring data
            try:
                if unified_client.ispm_client:
                    spatial_query = SpatialQuery(component_id=model_id)
                    result['ispm_monitoring'] = unified_client.ispm_client.query(spatial=spatial_query)
            except Exception as e:
                result['ispm_monitoring'] = {'error': str(e)}
            
            query_results = result
        else:
            # Demo mode: create synthetic results
            result = {
                'model_id': model_id,
                'stl_model': {'name': f'Model {model_id}', 'vertices': 1000} if 'stl' in selected_sources else None,
                'hatching_layers': [{'layer': i, 'paths': 10} for i in range(10)] if 'hatching' in selected_sources else None,
                'laser_parameters': {'count': 500, 'power_range': (100, 300)} if 'laser' in selected_sources else None,
                'ct_scan': {'defects': 5, 'density_range': (0.8, 1.0)} if 'ct' in selected_sources else None,
                'ispm_monitoring': {'samples': 1000, 'temp_range': (500, 1200)} if 'ispm' in selected_sources else None
            }
            query_results = result
        
        progress_bar.value = 100
        
        # Update results display
        update_results_display()
        
        # Update status
        status_text.value = "<span style='color: green;'>✅ Query completed successfully</span>"
        
        # Update summary
        # Helper function to safely get count from result data
        def get_count(data):
            """Get count from data, handling both dict and QueryResult objects."""
            if data is None:
                return 0
            if QueryResult and isinstance(data, QueryResult):
                # QueryResult has points attribute
                return len(data.points) if hasattr(data, 'points') and data.points else 0
            elif isinstance(data, dict):
                # Dictionary - try common count keys
                return data.get('count', data.get('samples', len(data) if isinstance(data, (list, dict)) else 0))
            elif isinstance(data, list):
                return len(data)
            else:
                return 1 if data else 0
        
        total_items = sum([
            get_count(result.get('stl_model')),
            get_count(result.get('hatching_layers')),
            get_count(result.get('laser_parameters')),
            get_count(result.get('ct_scan')),
            get_count(result.get('ispm_monitoring'))
        ])
        summary_text.value = f"""
        <p><b>Model:</b> {model_id[:8]}...</p>
        <p><b>Sources:</b> {', '.join(selected_sources)}</p>
        <p><b>Total Items:</b> {total_items}</p>
        """
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_text.value = f"<span style='color: red;'>Error: {str(e)}</span>"
        progress_bar.value = 0

def update_results_display():
    """Update the results display tabs."""
    global query_results
    
    # Clear all outputs
    with table_output:
        clear_output(wait=True)
        if query_results:
            # Create a simple table display
            html = "<h4>Query Results</h4><table border='1' style='border-collapse: collapse; width: 100%;'>"
            html += "<tr><th>Source</th><th>Status</th><th>Details</th></tr>"
            
            for source, data in query_results.items():
                if source == 'model_id':
                    continue
                # Check if data exists and is not empty
                if data is not None:
                    if QueryResult and isinstance(data, QueryResult):
                        # Handle QueryResult objects
                        point_count = len(data.points) if hasattr(data, 'points') and data.points else 0
                        signal_count = len(data.signals) if hasattr(data, 'signals') and data.signals else 0
                        if point_count > 0 or signal_count > 0:
                            html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>{point_count} points, {signal_count} signals</td></tr>"
                        else:
                            html += f"<tr><td>{source}</td><td style='color: orange;'>⚠</td><td>Query returned 0 points, {signal_count} signals (no data found)</td></tr>"
                    elif isinstance(data, dict):
                        if 'error' in data:
                            html += f"<tr><td>{source}</td><td style='color: red;'>Error</td><td>{data['error']}</td></tr>"
                        else:
                            details = ', '.join([f"{k}: {v}" for k, v in list(data.items())[:3]])
                            html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>{details}</td></tr>"
                    elif isinstance(data, list):
                        if len(data) > 0:
                            html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>{len(data)} items</td></tr>"
                        else:
                            html += f"<tr><td>{source}</td><td style='color: orange;'>⚠</td><td>Empty list (0 items found)</td></tr>"
                    else:
                        html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>Available</td></tr>"
                else:
                    # get_all_data() queries all sources, so None means no data found for this source
                    html += f"<tr><td>{source}</td><td style='color: gray;'>-</td><td>No data available (query returned None)</td></tr>"
            
            html += "</table>"
            display(HTML(html))
        else:
            display(HTML("<p>No results to display. Execute a query first.</p>"))
    
    with stats_output:
        clear_output(wait=True)
        if query_results:
            html = "<h4>Statistics</h4><ul>"
            for source, data in query_results.items():
                if source == 'model_id':
                    continue
                if data:
                    if QueryResult and isinstance(data, QueryResult):
                        # Handle QueryResult objects
                        point_count = len(data.points) if hasattr(data, 'points') and data.points else 0
                        signal_count = len(data.signals) if hasattr(data, 'signals') else 0
                        html += f"<li><b>{source}:</b> {point_count} points, {signal_count} signals</li>"
                    elif isinstance(data, list):
                        html += f"<li><b>{source}:</b> {len(data)} items</li>"
                    elif isinstance(data, dict):
                        if 'count' in data:
                            html += f"<li><b>{source}:</b> {data['count']} records</li>"
                        elif 'error' in data:
                            html += f"<li><b>{source}:</b> Error - {data['error']}</li>"
                        else:
                            html += f"<li><b>{source}:</b> Available</li>"
                    else:
                        html += f"<li><b>{source}:</b> Available</li>"
            html += "</ul>"
            display(HTML(html))
        else:
            display(HTML("<p>No statistics available.</p>"))
    
    with viz_output:
        clear_output(wait=True)
        display(HTML("<h4>Visualization</h4><p>Visualization will be available in future notebooks.</p>"))
    
    with export_output:
        clear_output(wait=True)
        if query_results:
            display(HTML("<h4>Export Options</h4><p>Export functionality will be available in future versions.</p>"))
        else:
            display(HTML("<p>No data to export.</p>"))

def clear_results(button):
    """Clear all results and reset widgets."""
    global query_results, query_dataframes
    query_results = {}
    query_dataframes = {}
    
    with table_output:
        clear_output()
    with stats_output:
        clear_output()
    with viz_output:
        clear_output()
    with export_output:
        clear_output()
    
    status_text.value = "Ready to query"
    summary_text.value = "No query executed yet"
    progress_bar.value = 0
    error_display.value = ""

def refresh_models(button):
    """Refresh the model list."""
    global models, model_options
    if stl_client and mongo_client:
        try:
            models = stl_client.list_models(limit=100)
            model_options = [("━━━ Choose a model ━━━", None)]
            model_options.extend([
                (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", 
                 m.get('model_id')) for m in models
            ])
            if len(model_options) == 1:
                model_options.append(("No models available", None))
            model_dropdown.options = model_options
            status_text.value = "<span style='color: green;'>✅ Models refreshed</span>"
        except Exception as e:
            status_text.value = f"<span style='color: red;'>Error: {str(e)}</span>"
    else:
        status_text.value = "<span style='color: orange;'>⚠️ MongoDB not available - using demo models</span>"

# Connect button events
query_button.on_click(execute_query)
clear_button.on_click(clear_results)
refresh_button.on_click(refresh_models)

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Understanding QueryResult Structure

The `QueryResult` object contains the data returned from queries. Let's explore its structure:


In [3]:
# Display QueryResult structure information
structure_info = """
## QueryResult Structure

A `QueryResult` object contains:

### Attributes:
- **`data`**: Dictionary containing source-specific data
- **`metadata`**: Dictionary with query metadata (model_id, sources, filters, etc.)
- **`source`**: String indicating the data source
- **`spatial_bounds`**: Tuple of bounding box `((x_min, y_min, z_min), (x_max, y_max, z_max))`
- **`temporal_range`**: Tuple of time range `(start_time, end_time)`

### Data Structure by Source:

#### Hatching Data:
- **Points**: Array of (x, y, z) coordinates
- **Layers**: Layer information
- **Paths**: Hatching path data

#### Laser Parameters:
- **Points**: Array of (x, y, z) coordinates  
- **Signals**: Dictionary with 'power', 'speed', 'energy_density', etc.

#### CT Scan Data:
- **Points**: Array of (x, y, z) coordinates
- **Signals**: Dictionary with 'density', 'defect_flag', etc.

#### ISPM Monitoring:
- **Points**: Array of (x, y, z) coordinates
- **Signals**: Dictionary with 'temperature', 'sensor_id', 'timestamp', etc.

### Example Usage:

```python
# Query data
result = unified_client.query(
    model_id="my_model",
    sources=['hatching', 'laser'],
    spatial_bbox=((-50, -50, 0), (50, 50, 100)),
    temporal_range=(0, 1000)
)

# Access data
points = result.points  # Array of (x, y, z) coordinates
signals = result.signals  # Dictionary of signal arrays
metadata = result.metadata  # Query metadata
```
"""

display(Markdown(structure_info))



## QueryResult Structure

A `QueryResult` object contains:

### Attributes:
- **`data`**: Dictionary containing source-specific data
- **`metadata`**: Dictionary with query metadata (model_id, sources, filters, etc.)
- **`source`**: String indicating the data source
- **`spatial_bounds`**: Tuple of bounding box `((x_min, y_min, z_min), (x_max, y_max, z_max))`
- **`temporal_range`**: Tuple of time range `(start_time, end_time)`

### Data Structure by Source:

#### Hatching Data:
- **Points**: Array of (x, y, z) coordinates
- **Layers**: Layer information
- **Paths**: Hatching path data

#### Laser Parameters:
- **Points**: Array of (x, y, z) coordinates  
- **Signals**: Dictionary with 'power', 'speed', 'energy_density', etc.

#### CT Scan Data:
- **Points**: Array of (x, y, z) coordinates
- **Signals**: Dictionary with 'density', 'defect_flag', etc.

#### ISPM Monitoring:
- **Points**: Array of (x, y, z) coordinates
- **Signals**: Dictionary with 'temperature', 'sensor_id', 'timestamp', etc.

### Example Usage:

```python
# Query data
result = unified_client.query(
    model_id="my_model",
    sources=['hatching', 'laser'],
    spatial_bbox=((-50, -50, 0), (50, 50, 100)),
    temporal_range=(0, 1000)
)

# Access data
points = result.points  # Array of (x, y, z) coordinates
signals = result.signals  # Dictionary of signal arrays
metadata = result.metadata  # Query metadata
```


## Summary

Congratulations! You've learned how to query data from the AM-QADF warehouse.

### Key Takeaways

1. **UnifiedQueryClient**: Single interface for querying all data sources
2. **Spatial Filtering**: Filter data by bounding box or full model
3. **Temporal Filtering**: Filter by layers or time ranges
4. **Multi-Source Queries**: Query multiple sources in a single operation
5. **QueryResult Structure**: Understand the data structure returned from queries

### Next Steps

Proceed to:
- **02_Voxel_Grid_Creation.ipynb** - Learn to create voxel grids from queried data
- **03_Signal_Mapping_Fundamentals.ipynb** - Learn to map signals to voxel grids

### Related Resources

- Query Module Documentation: `../docs/AM_QADF/05-modules/query.md`
- API Reference: `../docs/AM_QADF/06-api-reference/query-api.md`
- Examples: `../examples/`
